In [1]:
from time import time
import tensorflow as tf
from helper import *
from models import *
from cleaning import *
from embedding import *
from nn_models import *
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

Reading twitter - 1grams ...
Reading twitter - 2grams ...


C:\Users\ahmed\anaconda3\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in
C:\Users\ahmed\anaconda3\lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [2]:
simple_nn = keras.models.load_model("simple_nn")
cnn = keras.models.load_model("CNN")
multi_cnn = keras.models.load_model("multi_cnn_glove_trained")

In [3]:
#load clean data 
with open('clean_pos_train_with_stopWords.txt',"r",encoding = "utf8") as file:
    pos = file.read().split('\n')
pos = pd.DataFrame({'tweet' : pos})[:len(pos)-1]

with open('clean_neg_train_with_stopWords.txt',"r",encoding = "utf8") as file:
    neg = file.read().split('\n')
neg = pd.DataFrame({'tweet' : neg})[:len(neg)-1]

pos['target'] = 1
neg['target'] = 0

#concat pos and neg to form full training set
train= pd.concat([pos, neg])
X_train = train.tweet
y_train = train.target

In [4]:
with open('clean_test_with_stopWords.txt',"r") as file:
    X_test = file.read().split('\n')
X_test = pd.DataFrame({'tweet' : X_test})[:len(X_test)-1]

In [5]:
X_train_tok,X_test_tok,word_dict , vocab_size = data_tokenizer(X_train,X_test.tweet)

In [ ]:
y_pred_simple_nn = simple_nn.predict(X_test_tok)
y_pred_simple_nn[y_pred_simple_nn >= 0.5 ] = 1
y_pred_simple_nn[y_pred_simple_nn < 0.5 ] = 0

y_pred_cnn = cnn.predict(X_test_tok)
y_pred_cnn[y_pred_cnn >= 0.5 ] = 1
y_pred_cnn[y_pred_cnn < 0.5 ] = 0

y_pred_multi_cnn = multi_cnn.predict(X_test_tok)
y_pred_multi_cnn[y_pred_multi_cnn >= 0.5 ] = 1
y_pred_multi_cnn[y_pred_multi_cnn < 0.5 ] = 0

In [31]:
y_pred = np.array(y_pred_simple_nn + y_pred_cnn + y_pred_multi_cnn, dtype = 'int64')
final_pred = y_pred.copy()
final_pred[y_pred >= 2] = 1
final_pred[y_pred < 2] = -1

In [32]:
ids_test = np.arange(1, len(y_pred)+1)
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in .csv format for submission to Kaggle or AIcrowd
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w',newline='') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})
            
OUTPUT_PATH = 'Voting.csv' # TODO: fill in desired name of output file for submission
create_csv_submission(ids_test, final_pred, OUTPUT_PATH)